In [96]:
import random
import networkx as nx
import itertools
import numpy as np
import pandas as pd

In [82]:
def mc_upper_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "chromatic_number" integer upper bound on the maximum clique number
	"""
	answ = nx.algorithms.coloring.greedy_color(G)
	chromatic_number = list(set(list(answ.values())))
	return len(chromatic_number)

# Funzione
## Nel markdown seguente viene data una spiegazione del funzionamento

In [83]:
# Funzione ricorsiva che sceglie un nodo random che non sia v o un nodo nella lista di nodi non utilizzabili
# Ritorna "None" se sono già stati controllati tutti i nodi così da bloccare il ciclo
def random_node(nodes, v, no_node_list):
    node = random.choice(list(nodes))
    if len(no_node_list) >= len(nodes)-1:
        return None
    elif node != v and node not in no_node_list:
        return node
    else:
        return random_node(nodes, v, no_node_list)

In [84]:
# Crea un grafo con "num_nodes" = numero nodi, con "dim_cli" nodi di cricca e "num_cli" cricche, poi collega i nodi delle cricche
def create_graph_with_rand_clique(num_nodes, dim_cli, num_cli, random_clique_dimension):
    g = nx.Graph()
    cliques_nodes = []
    for i in range(num_nodes):
        g.add_node(i)
    nodes = list(g.nodes())
    for k in range(num_cli):
        if random_clique_dimension:
            clique = random.sample(nodes, random.randint(2,dim_cli))
        else:
            clique = random.sample(nodes, dim_cli)
        cliques_nodes.append(sorted(clique))
        nodes = [a for a in nodes if a not in clique]
        combinations = itertools.combinations(clique,2)
        g.add_edges_from(combinations)
    return g, cliques_nodes, nodes

In [85]:
# "num_node" = numero nodi cricca, 
# "ex_node" nodi non appartenenti alla cricca, 
# "density" nuemro di terazioni per creare archi
# "add_edges_between_cliques" il nome spiega, meglio mettere a True
# "random_clique_dimension" setta la dim della cricca random con massimo "num_node", per ora settare a False
def graph_with_clique(num_node, ex_node, num_clique, density, add_edges_between_cliques, random_clique_dimension):
    g, clique_nodes, list_ex_nodes = create_graph_with_rand_clique(num_node*num_clique+ex_node, num_node, num_clique, random_clique_dimension)
    total_nodes = len(g.nodes())
    ub = mc_upper_bound(g)
    ld = (density-5)/100 #lower density
    ud = (density+5)/100 #upper density
    for i in list_ex_nodes:
        links = random.randint(int(total_nodes*ld), int(total_nodes*ud)) #numero di iterazioni
        no_nodes_1 = []
        for _ in range(links):
            node = random_node(g.nodes, i, no_nodes_1) #scelgo nodo
            if node is None: #Se ho usato tutti i nodi fermo
                break
            g.add_edge(i, node) #aggiunge arco
            new_ub = mc_upper_bound(g) #ricalcola upper bound
            if new_ub > ub: #rimuove arco se necessario
                g.remove_edge(i, node)
            no_nodes_1.append(node)
    if add_edges_between_cliques:
        for cli in clique_nodes:
            ok_nodes = [a for a in g.nodes() if a not in cli]
            for n in cli:
                links = random.randint(int(total_nodes*ld), int(total_nodes*ud)) #numero di iterazioni
                no_nodes_2 = []
                for _ in range(links):
                    node = random_node(ok_nodes, n, no_nodes_2) #scelgo nodo
                    if node is None: #Se ho usato tutti i nodi fermo
                        break
                    g.add_edge(n, node) #aggiunge arco
                    new_ub = mc_upper_bound(g) #ricalcola upperbound
                    if new_ub > ub: #rimuove arco se necessario
                        g.remove_edge(n, node)
                    no_nodes_2.append(node)
    return g, clique_nodes

# Importante
### La funzione decide quante volte iterare una funzione una funzione che crea archi.
### Per creare un arco da un vertice V: viene scelto un nodo random V1, prova a creare l'arco, se aumenta l'upper bound rimuove l'arco e aggiunge il nodo V1 in una lista provvisoria di nodi non utilizzabili con il verticer V. 
### L'iterazione appena spiegata viene eseguita un numero di volte random pari a: numero totale dei nodi*density/100 
### Il codice sottostante serve per provare con vari valori di densità quale è più adatto a ottenere un grafo con una certa densità dato la dimensione della cricca e il numero di nodi esterni

In [98]:
dens = [i for i in range(70,100,10)] #scegliere il range di valori di density
dim_cli = 30
dim_ext = 100
n_cli = 1
print(f"Grafo con {n_cli} cricca/cricche, con {dim_cli} nodi della cricca e {dim_ext} nodi esterni")
for density in dens:
    G, cli = graph_with_clique(dim_cli, dim_ext, n_cli, density, True, False)
    d = nx.density(G)
    print(f"Per il valore di density={density} la densità del grafo è {d}")

Grafo con 1 cricca/cricche, con 30 nodi della cricca e 100 nodi esterni
Per il valore di density=70 la densità del grafo è 0.8657125819916518
Per il valore di density=80 la densità del grafo è 0.9354800238521169
Per il valore di density=90 la densità del grafo è 0.9588550983899821


In [99]:
G, cli = graph_with_clique(dim_cli, dim_ext, n_cli, 75, True, False)

In [100]:
degrees = dict(G.degree())
print(f"Rapporto grado minimo:{min(degrees.values())/len(G)}")
print(f"Rapporto grado massimo:{max(degrees.values())/len(G)}")

Rapporto grado minimo:0.7692307692307693
Rapporto grado massimo:0.9692307692307692


# Funzione di assegnazione pesi
### Input grafo e i due dataframe di probabilità
### Output un dict che contiene vertice:[w00,w01]
### Viene sviluppata così in maniera da poter estrarre più set di pesi per lo stesso grafo, il dict verrà passato poi alla funzione di assegnazione pesi

In [368]:
def assegna_pesi_casuali(grafo, df0, df1):
    # Calcola il numero totale di nodi nel grafo
    num_nodi = grafo.number_of_nodes()
    df_weights = {}
    
    # Itera su ogni nodo del grafo
    for nodo in grafo.nodes():
        # Calcola il rapporto grado/numero di nodi per il nodo attuale
        grado = grafo.degree[nodo]
        rapporto = round(grado / num_nodi, 3)
        
        # Trova la riga nel dataframe w00 corrispondente al rapporto più vicino
        if rapporto in df0.index:
            riga0 = df0.loc[rapporto]
        else:
            idx_rapporto_piu_vicino0 = pd.DataFrame(df0.index - rapporto).abs().idxmin()
            rapporto_piu_vicino0 = df0.index[idx_rapporto_piu_vicino0]
            riga0 = df0.loc[rapporto_piu_vicino0[0]]
        # Trova la riga nel dataframe w01 corrispondente al rapporto più vicino
        if rapporto in df1.index:
            riga1 = df1.loc[rapporto]
        else:
            idx_rapporto_piu_vicino1 = pd.DataFrame(df0.index - rapporto).abs().idxmin()
            rapporto_piu_vicino1 = df0.index[idx_rapporto_piu_vicino1]
            riga1 = df1.loc[rapporto_piu_vicino1[0]]
        
        # Estrai la probabilità dalla riga e normalizzala se necessario
        probabilita0 = riga0.values
        pesi0 = riga0.index
        probabilita1 = riga1.values
        pesi1 = riga1.index
        
        # Scegli un peso casualmente secondo la distribuzione di probabilità
        peso_scelto0 = np.random.choice(pesi0, p=probabilita0)
        peso_scelto1 = np.random.choice(pesi1, p=probabilita1)
        
        #aggiungo al dataframe il nodo e peso
        df_weights[nodo] = [peso_scelto0, peso_scelto1]
    
    return df_weights

## Test

In [302]:
matrice_w00 = pd.read_csv('./prob_w00.csv', index_col=0)
matrice_w01 = pd.read_csv('./prob_w00.csv', index_col=0)

In [129]:
G, cli = graph_with_clique(dim_cli, dim_ext, n_cli, 75, True, False)

In [369]:
a = assegna_pesi_casuali(G, matrice_w00, matrice_w01)

0.982
0.0       1.0
0.0003    0.0
0.0004    0.0
0.0005    0.0
0.0006    0.0
         ... 
0.9945    0.0
0.9958    0.0
0.9968    0.0
0.9976    0.0
1.0       0.0
Name: 0.977, Length: 340, dtype: float64
